In [28]:
import os
import logging
import math
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [29]:
STOPWORDS = set(stopwords.words('english'))
LEMMATIZER = WordNetLemmatizer()

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
def load_text_files(folder_path):
    data = {}
    doc_id_to_filename = {}
    doc_id = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                data[doc_id] = file.read()
                doc_id_to_filename[doc_id] = filename
                logging.info(f"Loaded file: {filename} with doc_id: {doc_id}")
                doc_id += 1
    return data, doc_id_to_filename


In [42]:
# Removes special characters from text, tokenizes it, eliminates stop words, and lemmatizes it.
def tokenize(text):
  tokens = text.lower()
  text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
  tokens = text.split()
  cleaned_tokens = [LEMMATIZER.lemmatize(word) for word in tokens if word not in STOPWORDS]
  return cleaned_tokens


In [33]:
def term_frequency(term, document):
    return document.count(term) / len(document)

In [34]:
def inverse_document_frequency(term, all_documents):
    num_docs_containing_term = sum(1 for doc in all_documents if term in doc)
    return math.log(len(all_documents) / (1 + num_docs_containing_term))

In [35]:
def compute_tfidf(document, all_documents, vocab):
    tfidf_vector = []
    for term in vocab:
        tf = term_frequency(term, document)
        idf = inverse_document_frequency(term, all_documents)
        tfidf_vector.append(tf * idf)
    return np.array(tfidf_vector)

In [36]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2) if norm_vec1 * norm_vec2 != 0 else 0

In [43]:
def main():
    folder_path = "/content/drive/MyDrive/TECH 400 Information Retrieval/Docs"

    docs, doc_id_to_filename = load_text_files(folder_path)

    queries = ["balanced diet",
               "good health",
               "technology",
               "climate change",
               "education",
               "ecosystem",
               ]

    tokenized_docs = [tokenize(doc) for doc in docs.values()]

    vocab = sorted(set(word for doc in tokenized_docs for word in doc))
    print(logging.info(f"Vocabulary size: {len(vocab)}"))

    doc_tfidf_vectors = [compute_tfidf(doc, tokenized_docs, vocab) for doc in tokenized_docs]

    results = []
    for query in queries:
        tokenized_query = tokenize(query)
        query_tfidf_vector = compute_tfidf(tokenized_query, tokenized_docs, vocab)

        similarities = []
        for doc_id, doc_vector in enumerate(doc_tfidf_vectors):
            similarity = cosine_similarity(query_tfidf_vector, doc_vector)
            similarities.append((doc_id, similarity))

        similarities.sort(key=lambda x: x[1], reverse=True)

        results.append((query, similarities))

        print(results)

    path = "/content/drive/MyDrive/TECH 400 Information Retrieval/result"
    output_file = os.path.join(path, "results_Nisha.txt")
    with open(output_file, 'w', encoding='utf-8') as f:
        for query, similarities in results:
            f.write(f"Query: {query}\n")
            for doc_id, similarity in similarities:
                filename = doc_id_to_filename[doc_id]
                f.write(f"  Document: {filename}, Similarity: {similarity:.4f}\n")
            f.write("\n")

    logging.info(f"Results written to {output_file}")

if __name__ == "__main__":
    main()

None
[('balanced diet', [(9, 0.2542962722727766), (1, 0.19916160642137054), (0, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)])]
[('balanced diet', [(9, 0.2542962722727766), (1, 0.19916160642137054), (0, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]), ('good health', [(9, 0.17447725662825692), (5, 0.03239399707527137), (1, 0.0320703059931084), (7, 0.030545118637625747), (0, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (6, 0.0), (8, 0.0)])]
[('balanced diet', [(9, 0.2542962722727766), (1, 0.19916160642137054), (0, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]), ('good health', [(9, 0.17447725662825692), (5, 0.03239399707527137), (1, 0.0320703059931084), (7, 0.030545118637625747), (0, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (6, 0.0), (8, 0.0)]), ('technology', [(0, 0.24702023445481192), (6, 0.22673008897439073), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (7, 0.0), (8, 0.0), (9, 0.0)])]
[('balanced diet', [